# Cleaning Public Support data

## Importing necessary libraries

In [1]:
pip install pandas matplotlib seaborn wordcloud

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 95.5 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 83.2 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 kB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.0/459.0 kB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 498.1/498.1 kB 79.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 84.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 139.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.0/296.0 kB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 kB 104.3 MB/s eta 0:00:00

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime,date, timedelta

## Loading the data

In [3]:
slack = pd.read_csv('../sources/support-channels.csv')


## Discover

In [4]:
print('Shape of slack dataframe before cleaning:', slack.shape)

Shape of slack dataframe before cleaning: (481, 14)


In [5]:
slack.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481 entries, 0 to 480
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Channel_ID        481 non-null    object
 1   Channel_Slug      481 non-null    object
 2   Timestamp         481 non-null    object
 3   Timestamp_Thread  368 non-null    object
 4   User_ID           481 non-null    object
 5   Full_Name         470 non-null    object
 6   Email             481 non-null    object
 7   Permalink         481 non-null    object
 8   Text              481 non-null    object
 9   Text_raw          481 non-null    object
 10  Slack_username    481 non-null    object
 11  Team_ID           481 non-null    object
 12  Team_Name         481 non-null    object
 13  Is_Bot            481 non-null    bool  
dtypes: bool(1), object(13)
memory usage: 49.4+ KB


**Creating 2 new columns**

In [6]:
slack['Is_a_question'] = np.where(slack['Timestamp_Thread'].isnull(), 1, 0)

In [7]:
support_agents = ['1','5301']

slack['Is_agent'] = np.where(slack['User_ID'].isin(support_agents), 1, 0)

**Encoding necessary columns**

In [8]:
slack['Is_Bot'] = np.where(slack['Is_Bot'] == True, 1, 0)

**Converting timestamp columns**

In [9]:
slack['Datetime'] = pd.to_datetime(slack['Timestamp'])
slack['Datetime_Thread'] = pd.to_datetime(slack['Timestamp_Thread'])

**Creating 2 dataframes: questions and answers**

In [10]:
Q_df = slack[slack['Is_a_question'] == 1]
A_df = slack[slack['Is_a_question'] == 0]

**Working on Answers dataframe**

In [11]:
answers = A_df.groupby(['User_ID','Datetime'])[['Text']]

In [12]:
df_answers = pd.DataFrame(answers.sum().reset_index())

In [13]:
df_answers.head()

,User_ID,Datetime,Text
0,U01KGAER1TM,2022-11-04 17:02:51,No se quiere usar un tercero para las fotos
1,U01KGAER1TM,2022-11-04 17:04:57,digamos que son imagenes de usuarios en donde ...
2,U01KGAER1TM,2022-11-04 17:05:02,comentarios y likes
3,U01KGAER1TM,2022-11-04 17:06:30,y la base de datos no solo contiene las imagen...
4,U01KGAER1TM,2022-11-04 17:26:59,https://isn365.com/


In [14]:
df_answers['Diff_in_Seconds'] = (df_answers.sort_values('Datetime').groupby('User_ID').Datetime.diff())

In [15]:
df_answers['Diff_in_Seconds'] = df_answers['Diff_in_Seconds'].fillna(pd.Timedelta(seconds=0))

In [16]:
df_answers['Diff_in_Seconds'] = df_answers['Diff_in_Seconds']/np.timedelta64(1,'s')

In [17]:
df_answers['diff_abs'] = df_answers.Diff_in_Seconds.abs()

In [18]:
df_answers['same_author'] = df_answers['User_ID'].ne(df_answers['User_ID'].shift().bfill()).astype(int)

In [19]:
def create_AnswerId(df):
    for group in df.groupby(['User_ID']):
        df['messageId'] = df.diff_abs.gt(300).cumsum() + 1 + df.same_author.cumsum()
    return df
create_AnswerId(df_answers)

/tmp/ipykernel_1120/3464986642.py:2: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for group in df.groupby(['User_ID']):


,User_ID,Datetime,Text,Diff_in_Seconds,diff_abs,same_author,messageId
0,U01KGAER1TM,2022-11-04 17:02:51,No se quiere usar un tercero para las fotos,0.0,0.0,0,1
1,U01KGAER1TM,2022-11-04 17:04:57,digamos que son imagenes de usuarios en donde ...,126.0,126.0,0,1
2,U01KGAER1TM,2022-11-04 17:05:02,comentarios y likes,5.0,5.0,0,1
3,U01KGAER1TM,2022-11-04 17:06:30,y la base de datos no solo contiene las imagen...,88.0,88.0,0,1
4,U01KGAER1TM,2022-11-04 17:26:59,https://isn365.com/,1229.0,1229.0,0,2
...,...,...,...,...,...,...,...
363,UU409472Q,2022-11-11 00:32:33,No te preocupes no es algo que afecte,632178.0,632178.0,0,190
364,UU409472Q,2022-11-11 01:48:09,https://www.notion.so/4geeksacademy/How-to-del...,4536.0,4536.0,0,191
365,UU409472Q,2022-11-11 14:33:32,"Meri , sigue estos pasos https://www.notion.so...",45923.0,45923.0,0,192
366,UU409472Q,2022-11-11 15:27:50,:anguished: intenta ir al dashboard a ver,3258.0,3258.0,0,193


In [20]:
df_answers.shape

(368, 7)

In [21]:
A_df.shape

(368, 18)

**Questions dataframe**

In [22]:
questions = Q_df.groupby(['User_ID','Datetime'])[['Text']]

In [23]:
df_questions = pd.DataFrame(questions.sum().reset_index())

In [24]:
df_questions.head()

,User_ID,Datetime,Text
0,U01KGAER1TM,2022-11-04 16:14:32,"Como estan, tengo un problema, ocupo subir una..."
1,U01SJ480RBR,2022-11-07 16:59:39,Hello
2,U01SM5J4MMG,2022-11-07 21:15:26,"Buenas tardes, intenta poniendo solo git push"
3,U02N1P8CV6W,2022-10-31 22:55:36,"Hola chicos, he estado intentando llevar a pro..."
4,U02NE11UHNC,2022-11-03 22:28:10,have someone imported custom fonts (no google ...


In [25]:
df_questions.shape

(112, 3)

In [26]:
df_questions['Diff_in_Seconds'] = (df_questions.sort_values('Datetime').groupby('User_ID').Datetime.diff())

In [27]:
df_questions['Diff_in_Seconds'] = df_questions['Diff_in_Seconds'].fillna(pd.Timedelta(seconds=0))

In [28]:
df_questions['Diff_in_Seconds']=df_questions['Diff_in_Seconds']/np.timedelta64(1,'s')

In [29]:
df_questions['diff_abs'] = df_questions.Diff_in_Seconds.abs()

In [30]:
df_questions['same_author'] = df_questions['User_ID'].ne(df_questions['User_ID'].shift().bfill()).astype(int)

In [31]:
def create_QuestionId(df):
    for group in df.groupby(['User_ID']):
        df['messageId'] = df['diff_abs'].gt(300).cumsum() + 1 + df.same_author.cumsum()
    return df

create_QuestionId(df_questions)

/tmp/ipykernel_1120/2400834575.py:2: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for group in df.groupby(['User_ID']):


,User_ID,Datetime,Text,Diff_in_Seconds,diff_abs,same_author,messageId
0,U01KGAER1TM,2022-11-04 16:14:32,"Como estan, tengo un problema, ocupo subir una...",0.0,0.0,0,1
1,U01SJ480RBR,2022-11-07 16:59:39,Hello,0.0,0.0,1,2
2,U01SM5J4MMG,2022-11-07 21:15:26,"Buenas tardes, intenta poniendo solo git push",0.0,0.0,1,3
3,U02N1P8CV6W,2022-10-31 22:55:36,"Hola chicos, he estado intentando llevar a pro...",0.0,0.0,1,4
4,U02NE11UHNC,2022-11-03 22:28:10,have someone imported custom fonts (no google ...,0.0,0.0,1,5
...,...,...,...,...,...,...,...
107,U04A6KV066M,2022-11-08 20:08:45,"Hi all,",0.0,0.0,1,89
108,U04A6KV066M,2022-11-08 20:13:24,Hi all. I am trying to split my view to where ...,279.0,279.0,0,89
109,U04A6KV066M,2022-11-08 21:58:27,Should we be doing anything with “commit” and ...,6303.0,6303.0,0,90
110,U6MR8LG4Q,2022-11-05 03:11:30,Hey @betomasia12 No need to make the pictures ...,0.0,0.0,1,91


In [32]:
df_questions.shape

(112, 7)

In [33]:
df_questions.head(5)

,User_ID,Datetime,Text,Diff_in_Seconds,diff_abs,same_author,messageId
0,U01KGAER1TM,2022-11-04 16:14:32,"Como estan, tengo un problema, ocupo subir una...",0.0,0.0,0,1
1,U01SJ480RBR,2022-11-07 16:59:39,Hello,0.0,0.0,1,2
2,U01SM5J4MMG,2022-11-07 21:15:26,"Buenas tardes, intenta poniendo solo git push",0.0,0.0,1,3
3,U02N1P8CV6W,2022-10-31 22:55:36,"Hola chicos, he estado intentando llevar a pro...",0.0,0.0,1,4
4,U02NE11UHNC,2022-11-03 22:28:10,have someone imported custom fonts (no google ...,0.0,0.0,1,5


**Merge each dataframe to its previous columns**

In [34]:
df_answers = df_answers.merge(A_df, how = 'left', left_on = ['User_ID', 'Datetime', 'Text'],
    right_on = ['User_ID', 'Datetime', 'Text']).drop(['Diff_in_Seconds','diff_abs','same_author','Text_raw'], axis=1)

In [35]:
df_answers['Response_time'] = df_answers['Datetime'] - df_answers['Datetime_Thread']

In [36]:
df_answers.shape

(368, 19)

In [37]:
df_questions = df_questions.merge(Q_df, how = 'left', left_on = ['User_ID', 'Datetime', 'Text'],
    right_on = ['User_ID', 'Datetime', 'Text']).drop(['Diff_in_Seconds','diff_abs','same_author','Text_raw'], axis=1)

In [38]:
df_questions.shape

(112, 18)

**Merge text and timestamps in rows that have the same messageId**

In [39]:
df_answers['Text'] = df_answers.groupby(['messageId'])['Text'].transform(lambda x : ' '.join(x))
df_questions['Text'] = df_questions.groupby(['messageId'])['Text'].transform(lambda x : ' '.join(x))

In [40]:
df_questions.dropna(axis=1, how='all', inplace=True)
df_questions['Timestamp'] = df_questions.groupby(['messageId'])['Timestamp'].transform(lambda x : ','.join(map(str, x)))

In [41]:
#rename to ids in both dataframes
df_questions.rename(columns={"Timestamp": "Question_ID", "Text":"Question_Text"}, inplace=True)
df_answers.rename(columns={"Timestamp": "Answer_ID", "Timestamp_Thread": "Key_to_Question_ID",
                    "User_ID":"Answer_User_ID","Full_Name":"Answer_Full_Name","Email":"Answer_email","Text":"Answer_Text","Is_agent":"Answer_from_Agent",
                    "Datetime":"Answer_Datetime", "Datetime_Thread":"Answer_Dt_Thread"},inplace=True)

In [42]:
#Drop duplicates
df_questions = df_questions.drop_duplicates(subset=["Question_Text","Question_ID"],keep='first')
df_answers = df_answers.drop_duplicates(subset=["Answer_Text"],keep='first')

In [43]:
#create a list from df3 question_id column
question_ids_list = df_questions['Question_ID'].tolist()

In [44]:
def id_autocompletion(search):
    for id in question_ids_list:
        if search in id:
            return id
    
    return None

In [45]:
df_answers['Key_to_Question_ID'] =  df_answers['Key_to_Question_ID'].apply(id_autocompletion)

In [46]:
df_answers['Key_to_Question_ID'] =  np.where(df_answers['Key_to_Question_ID'].isnull(), str(df_answers['Answer_Dt_Thread']), df_answers['Key_to_Question_ID'])

In [47]:
df_answers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192 entries, 0 to 366
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype          
---  ------              --------------  -----          
 0   Answer_User_ID      192 non-null    object         
 1   Answer_Datetime     192 non-null    datetime64[ns] 
 2   Answer_Text         192 non-null    object         
 3   messageId           192 non-null    int64          
 4   Channel_ID          192 non-null    object         
 5   Channel_Slug        192 non-null    object         
 6   Answer_ID           192 non-null    object         
 7   Key_to_Question_ID  192 non-null    object         
 8   Answer_Full_Name    189 non-null    object         
 9   Answer_email        192 non-null    object         
 10  Permalink           192 non-null    object         
 11  Slack_username      192 non-null    object         
 12  Team_ID             192 non-null    object         
 13  Team_Name           192 non-null   

In [48]:
#verifying it worked
df_answers[['Answer_User_ID','Answer_ID','Key_to_Question_ID', 'Answer_Datetime', 'Answer_Dt_Thread']]

,Answer_User_ID,Answer_ID,Key_to_Question_ID,Answer_Datetime,Answer_Dt_Thread
0,U01KGAER1TM,11/4/2022 17:02:51,11/4/2022 16:14:32,2022-11-04 17:02:51,2022-11-04 16:14:32
4,U01KGAER1TM,11/4/2022 17:26:59,11/4/2022 16:14:32,2022-11-04 17:26:59,2022-11-04 16:14:32
6,U01KGAER1TM,11/4/2022 17:34:48,11/4/2022 16:14:32,2022-11-04 17:34:48,2022-11-04 16:14:32
10,U01SM5J4MMG,11/8/2022 14:09:39,11/7/2022 21:15:26,2022-11-08 14:09:39,2022-11-07 21:15:26
12,U0230UQHWKS,11/4/2022 17:30:11,11/4/2022 16:14:32,2022-11-04 17:30:11,2022-11-04 16:14:32
...,...,...,...,...,...
361,UU409472Q,11/3/2022 16:55:38,11/3/2022 16:41:34,2022-11-03 16:55:38,2022-11-03 16:41:34
363,UU409472Q,11/11/2022 0:32:33,"11/10/2022 22:34:28,11/10/2022 22:34:56",2022-11-11 00:32:33,2022-11-10 22:34:56
364,UU409472Q,11/11/2022 1:48:09,11/11/2022 1:06:40,2022-11-11 01:48:09,2022-11-11 01:06:40
365,UU409472Q,11/11/2022 14:33:32,11/11/2022 13:55:22,2022-11-11 14:33:32,2022-11-11 13:55:22


In [49]:
#Saving to csv

df_questions.to_csv('../output/questions.csv')

df_answers.to_csv('../output/answers.csv')

In [51]:
final_df = pd.merge(df_questions, df_answers[['Answer_User_ID','Answer_Full_Name','Answer_email','Answer_from_Agent','Answer_Text','Answer_ID','Key_to_Question_ID',
            'Answer_Datetime','Answer_Dt_Thread','Response_time']], how = 'left', left_on = ['Question_ID'], right_on = ['Key_to_Question_ID'])

In [ ]:
final_df.to_csv('../output/final_df.csv')